TASK 5 – PIVOT TABLE + STACK / UNSTACK

Chuẩn bị dữ liệu

In [1]:
import pandas as pd

df_candidate = pd.read_csv("../data_clean/candidate_profile_clean.csv")
df_interview = pd.read_csv("../data_clean/interview_score_clean.csv")
df_salary = pd.read_csv("../data_clean/salary_offer_clean.csv")


Pivot điểm trung bình theo vị trí × vòng phỏng vấn

In [2]:
# Gộp dữ liệu cần thiết
pivot_data = pd.merge(
    df_interview,
    df_salary[["candidate_id", "position"]],
    on="candidate_id",
    how="left"
)

pivot_score = pd.pivot_table(
    pivot_data,
    values="score",
    index="position",
    columns="round",
    aggfunc="mean"
)

pivot_score


round,1,2,3
position,,,
Data Analyst,7.250000,6.5,7.333333
Dev,9.000000,6.5,6.000000
Python Dev,6.666667,6.0,7.666667
Python Developer,7.333333,5.5,6.666667


📝 Nhận xét:Một số vị trí có sự cải thiện điểm qua các vòng phỏng vấn.

Pivot mức lương đề nghị theo vị trí × kinh nghiệm

In [3]:
salary_exp = pd.merge(
    df_salary,
    df_candidate[["candidate_id", "experience_years"]],
    on="candidate_id",
    how="left"
)

pivot_salary = pd.pivot_table(
    salary_exp,
    values="offer_salary",
    index="position",
    columns="experience_years",
    aggfunc="mean"
)

pivot_salary


experience_years,1.0,2.0,3.0,4.0,5.0
position,,,,,
Data Analyst,20000000.0,NaN,15000000.0,20000000.0,20000000.0
Dev,25000000.0,NaN,NaN,NaN,NaN
Python Dev,20000000.0,15000000.0,15000000.0,NaN,NaN
Python Developer,NaN,NaN,NaN,20000000.0,NaN


📝 Nhận xét:Mức lương đề nghị có xu hướng tăng theo kinh nghiệm, nhưng không hoàn toàn tuyến tính.

Stack & Unstack bảng pivot

In [4]:
# Stack
interview_position = df_interview.merge(
    df_salary[["candidate_id", "position"]],
    on="candidate_id",
    how="left"
)


In [5]:
# unstack
pivot_score = pd.pivot_table(
    interview_position,
    index="position",
    columns="round",
    values="score",
    aggfunc="mean"
)

pivot_score


round,1,2,3
position,,,
Data Analyst,7.250000,6.5,7.333333
Dev,9.000000,6.5,6.000000
Python Dev,6.666667,6.0,7.666667
Python Developer,7.333333,5.5,6.666667


📝 Nhận xét:Stack/unstack giúp chuyển đổi cấu trúc dữ liệu để phân tích linh hoạt hơn.

Nhận xét nhóm ứng viên mạnh nhất

In [6]:
import pandas as pd

df = pd.read_csv("../data_clean/hr_full_data.csv")

df.columns


Index(['candidate_id', 'full_name', 'experience_years', 'avg_score',
       'max_score', 'min_score', 'interview_rounds', 'offer_salary',
       'position'],
      dtype='object')

In [8]:
# Tính điểm trung bình theo ứng viên
candidate_strength = df.groupby(
    ["candidate_id", "full_name", "position"]
).agg(
    avg_score=("avg_score", "mean"),
    avg_salary=("offer_salary", "mean"),
    experience=("experience_years", "mean")
).reset_index()

# Lọc nhóm ứng viên mạnh
strong_candidates = candidate_strength[
    (candidate_strength["avg_score"] >= 8) &
    (candidate_strength["avg_salary"].notna()) &
    (candidate_strength["experience"] >= 3)
]

# Sắp xếp theo điểm giảm dần
strong_candidates = strong_candidates.sort_values(
    by="avg_score",
    ascending=False
)

strong_candidates.head()


,candidate_id,full_name,position,avg_score,avg_salary,experience
0,UV100,Hoang Van E,Data Analyst,9.0,15000000.0,3.0


Dựa trên kết quả tổng hợp, nhóm ứng viên mạnh nhất là những ứng viên có điểm phỏng vấn trung bình từ 8 trở lên, kinh nghiệm từ 3 năm và được đề nghị mức lương tương đối cao. Trong đó, các vị trí như Data Analyst và Software Engineer có số lượng ứng viên mạnh nhiều hơn so với các vị trí khác. Điều này cho thấy các vị trí này không chỉ thu hút ứng viên chất lượng mà còn được doanh nghiệp ưu tiên về mức lương đề nghị.